### imports

In [ ]:
import random
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pykrx import stock

yf.pdr_override()

# Sorting $\beta$ of KOSPI

## beta example 1
- 베타 계산 및 출력 basic

In [ ]:
tickers = ['^KS11', '000547.KS'] # 키움증권 티커

all_data = {}
for ticker in tickers:
    all_data[ticker] = web.DataReader(ticker,
                                      start="2016-01-01",
                                      end='2021-12-31')

prices = pd.DataFrame({tic: data['Close'] for tic, data in all_data.items()})
ret = prices.pct_change().dropna()

In [ ]:
ret['intercept'] = 1
reg = sm.OLS(ret[['000547.KS']], ret[['^KS11', 'intercept']]).fit()
print(reg.params)

## beta of kospi

1. pykrx에서 koppi의 모든 티커 가져온다
2. DataReader로 불러올 수 있는 데이터 모두 커버한다
3. 코스피 지수와 회귀분석하여 모두 베타 구한다
4. 베타 상위 20, 하위 20개 종목들 출력하여 비교한다

In [ ]:
def return_name(market):
    '''get name from ticker'''
    Market = []
    for ticker in market:
        Value = stock.get_market_ticker_name(ticker)
        Market.append([Value,ticker])
 
    df = pd.DataFrame(Market,columns=['회사명','상장번호'])
    return df

KOSPI = stock.get_market_ticker_list(market="KOSPI")
kospi = return_name(KOSPI)

코스피 종목 티커 가져오기

In [ ]:
tickers_list = [f'{ticker}.KR' for ticker in kospi['상장번호']]
# tickers
len(tickers_list)

In [ ]:
tickers_list[:5]

In [ ]:
def calc_beta(*tickers):

    all_data = {}
    
    for ticker in tickers:
        print(ticker)
        ticker = ['^KS11', ticker]
        all_data[ticker] = web.DataReader(ticker,
                                          start="2016-01-01",
                                          end='2021-12-31')

        prices = pd.DataFrame({tic: data['Close'] for tic, data in all_data.items()})
        ret = prices.pct_change().dropna()


In [ ]:
all_data = {}
tickers = [f'{i}.KS' for i in kospi['상장번호']]

for ticker in tqdm(tickers):
    all_data[ticker] = web.DataReader(ticker,
                                      start="2016-01-01",
                                      end='2021-12-31', actions=False)

prices = pd.DataFrame({tic: data['Close'] for tic, data in all_data.items()})
ret = prices.pct_change().dropna()

In [180]:
len(all_data)

943

In [199]:
kospi_data = web.DataReader('^KS11',
                             start="2016-01-01",
                             end='2021-12-31', actions=False)
kospi_ret = kospi_data['Close'].pct_change().dropna()
kospi_ret # 코스피 데이터

[*********************100%***********************]  1 of 1 completed


Date
2016-01-05    0.006134
2016-01-06   -0.002642
2016-01-07   -0.010959
2016-01-08    0.006979
2016-01-11   -0.011879
                ...   
2021-12-24    0.004756
2021-12-27   -0.004276
2021-12-28    0.006898
2021-12-29   -0.008923
2021-12-30   -0.005225
Name: Close, Length: 1472, dtype: float64

In [186]:
prices = pd.DataFrame({tic: data['Close'] for tic, data in all_data.items()})

In [ ]:
ret = prices.dropna(axis=1).pct_change().dropna()
ret['^KS11'] = kospi_ret
ret['intercept'] = 1 # 코스피와 intercept 칼럼 넣기

In [219]:
reg = sm.OLS(ret[['000547.KS']], ret[['^KS11', 'intercept']]).fit() 
print(reg.params[0])

0.8745141815680655


In [201]:
ret['000547.KS']

Date
2016-01-05    0.030303
2016-01-06   -0.032086
2016-01-07   -0.022099
2016-01-08   -0.019774
2016-01-11   -0.002882
                ...   
2021-12-24   -0.033628
2021-12-27    0.009158
2021-12-28   -0.001815
2021-12-29   -0.001818
2021-12-30    0.009107
Name: 000547.KS, Length: 1472, dtype: float64

In [217]:
ret_col = ret.columns[:-2] # 개별 종목 ticker 까지 columns설정
ret_col

Index(['095570.KS', '006840.KS', '027410.KS', '138930.KS', '001460.KS',
       '001465.KS', '001040.KS', '079160.KS', '000120.KS', '011150.KS',
       ...
       '094280.KS', '093370.KS', '081660.KS', '005870.KS', '079980.KS',
       '005010.KS', '000540.KS', '000547.KS', '000545.KS', '003280.KS'],
      dtype='object', length=811)

In [220]:
beta_data = {}

for ticker in ret_col:
    reg = sm.OLS(ret[[ticker]], ret[['^KS11', 'intercept']]).fit()
    beta_data[ticker] = reg.params[0]

In [230]:
beta_data
beta_series=pd.Series(beta_data)
beta_series.sort_values()

010580.KS     0.033781
155900.KS     0.063669
168490.KS     0.127617
088980.KS     0.128645
017390.KS     0.138271
               ...    
003530.KS     1.562759
001230.KS     1.569811
001470.KS     5.738736
001745.KS    14.717517
105560.KS    15.050342
Length: 811, dtype: float64

In [269]:
beta_bottom20= beta_series.sort_values()[:20]
beta_top20 = beta_series.sort_values()[-20:]

In [265]:
beta_bottom20.index
beta_bottom20.values

ticker_name = []
for i in beta_bottom20.index:
    # print(i.split('.')[0])
    # print(stock.get_market_ticker_name(i.split('.')[0]))
    ticker_name.append(stock.get_market_ticker_name(i.split('.')[0]))

pd.DataFrame({'ticker': beta_bottom20.index,
             'name': ticker_name,
            'beta': beta_bottom20.values})

,ticker,name,beta
0,010580.KS,에스엠벡셀,0.033781
1,155900.KS,바다로19호,0.063669
2,168490.KS,한국패러랠,0.127617
3,088980.KS,맥쿼리인프라,0.128645
4,017390.KS,서울가스,0.138271
5,021820.KS,세원정공,0.143437
6,016710.KS,대성홀딩스,0.169078
7,023450.KS,동남합성,0.180261
8,016800.KS,퍼시스,0.205887
9,003465.KS,유화증권우,0.213789


 통신업과 섬유ㆍ의복, 의료정밀, 제약, 종이ㆍ목재, 비금속, 전기ㆍ가스 등이 저베타업종으로 분류된다

In [276]:
ticker_name = []
for i in beta_top20.index:
    # print(i.split('.')[0])
    # print(stock.get_market_ticker_name(i.split('.')[0]))
    ticker_name.append(stock.get_market_ticker_name(i.split('.')[0]))

pd.DataFrame({'ticker': beta_top20.index,
             'name': ticker_name,
            'beta': beta_top20.values}).sort_values(by='beta',ascending=False)

,ticker,name,beta
19,105560.KS,KB금융,15.050342
18,001745.KS,SK네트웍스우,14.717517
17,001470.KS,삼부토건,5.738736
16,001230.KS,동국제강,1.569811
15,003530.KS,한화투자증권,1.562759
14,071050.KS,한국금융지주,1.513819
13,006800.KS,미래에셋증권,1.503280
12,003470.KS,유안타증권,1.497533
11,009470.KS,삼화전기,1.483026
10,005070.KS,코스모신소재,1.471121


고베타주에는 금융 관련 종목이 많이보인다


https://www.mk.co.kr/dic/m/view.php?idx=6378
- 대개 고베타 상위 업종으로는 전기전자·금융·의료정밀 등이 있고, 저베타 상위 업종에는 화학·의약품·비금속광물·음식료 등 경기에 둔감한 방어주들이 대부분이다